In [120]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import spark.implicits._

import $ivy.$
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import spark.implicits._

In [121]:
val spark = SparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark: SparkSession = org.apache.spark.sql.SparkSession@6238a68b

In [122]:
val peopleFile = "/host/storage/people.csv"

val df = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(peopleFile)

peopleFile: String = "/host/storage/people.csv"
df: DataFrame = [firstname: string, lastname: string ... 2 more fields]

### Count all people with the same last name by every state from people.csv file and displays the result as a percentage for every state.

In [123]:
val countByState = df
    .groupBy("state")
    .agg(count("state").as("countByState"))

val countByLastNameAndState = df
    .groupBy("state", "lastname")
    .agg(count("lastname").as("countByLastNameAndState"))

val lastNamePercentageByState = countByLastNameAndState
    .join(countByState, "state")
    .withColumn(
        "lastNamePercentageByState",
        $"countByLastNameAndState" / $"countByState" * 100
    )
    .select("state", "lastname", "lastNamePercentageByState")

lastNamePercentageByState.show()

+----------+---------+-------------------------+
|     state| lastname|lastNamePercentageByState|
+----------+---------+-------------------------+
|California|   Watson|                     20.0|
|   Florida|    Baker|                     20.0|
|   Florida|    Myers|                     10.0|
|California|   Walker|                     20.0|
|California|   Batres|                     10.0|
|   Florida|    Baird|                     10.0|
|   Florida|    Jones|                     10.0|
|   Florida|Ratchford|                     10.0|
|   Florida|   Foster|                     10.0|
|California|  Johnson|                     10.0|
|   Florida| Harrison|                     30.0|
|California|   Turner|                     40.0|
+----------+---------+-------------------------+



countByState: DataFrame = [state: string, countByState: bigint]
countByLastNameAndState: DataFrame = [state: string, lastname: string ... 1 more field]
lastNamePercentageByState: DataFrame = [state: string, lastname: string ... 1 more field]

### Save results to different files for each state

In [124]:
lastNamePercentageByState
    .write
    .partitionBy("state")
    .json("/host/storage/peopleByLastnameFromState")

In [125]:
spark.stop()